# Goal is to look into using codeBERT
* Want to  tokenize the entire training dataset

Takeaways: yeah that works

In [1]:
import pandas as pd
import numpy as np
import pickle
from transformers import AutoTokenizer, AutoModel
from datasets import Dataset
from datasets import load_dataset
import datasets
import torch
from collections import Counter
import string
from scipy import spatial


c:\Users\halom\anaconda3\envs\test2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(torch.backends.cudnn.enabled)
print(torch.cuda.is_available()) #We have GPU on deck and ready
print(f"CUDA device: {torch.cuda.get_device_name(torch.cuda.current_device())}")

True
True
CUDA device: NVIDIA GeForce RTX 3060 Laptop GPU


In [3]:
dataset_dict = datasets.load_from_disk("./Dataset/CodeSearchCorpus/")

In [4]:
train_dataset = dataset_dict["train"]
train_dataset

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 1880853
})

In [5]:
train_dataset = train_dataset.select_columns(["whole_func_string", "language"])
# train_dataset_DF = train_dataset.to_pandas()

In [6]:
train_dataset[0]

{'whole_func_string': 'def train(train_dir, model_save_path=None, n_neighbors=None, knn_algo=\'ball_tree\', verbose=False):\n    """\n    Trains a k-nearest neighbors classifier for face recognition.\n\n    :param train_dir: directory that contains a sub-directory for each known person, with its name.\n\n     (View in source code to see train_dir example tree structure)\n\n     Structure:\n        <train_dir>/\n        ├── <person1>/\n        │   ├── <somename1>.jpeg\n        │   ├── <somename2>.jpeg\n        │   ├── ...\n        ├── <person2>/\n        │   ├── <somename1>.jpeg\n        │   └── <somename2>.jpeg\n        └── ...\n\n    :param model_save_path: (optional) path to save model on disk\n    :param n_neighbors: (optional) number of neighbors to weigh in classification. Chosen automatically if not specified\n    :param knn_algo: (optional) underlying data structure to support knn.default is ball_tree\n    :param verbose: verbosity of training\n    :return: returns knn classifie

In [ ]:
model_ckpt = "huggingface/CodeBERTa-small-v1" #Can/Should test different models
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [ ]:
# Load the model to the GPU. Mine is a 3060
device = torch.device("cuda")
model.to(device)

In [ ]:
#From Hugging Face Tutorials
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [ ]:
# num_rows = 50000
# filepath_pkl_obj = "./PickleObjects/"
# inverted_index_name = f"inverted_index_{num_rows}.pkl"
# tsed_name = f"train_subset_embeddings_dataset_{num_rows}.pkl"

# print(inverted_index_name, tsed_name)

In [ ]:
# np.random.seed(1)
# num_rows = 50000
# train_subset_indices = np.random.choice(len(train_dataset), num_rows, replace = False)
# train_dataset_subset = train_dataset.select(train_subset_indices)

# len(train_dataset_subset)

In [ ]:
# train_embeddings_dataset = train_dataset.map(
#         lambda x: {"embeddings": get_embeddings(x["whole_func_string"]).detach().cpu().numpy()[0]}
#     )

# train_embeddings_dataset.add_faiss_index(column="embeddings")

# with open(f'./pickleObjects/tsed_codeBERT_FULL.pkl', 'wb') as f:  # open a text file
#     pickle.dump(train_embeddings_dataset, f) # serialize the list
#     f.close()

In [ ]:
# train_embeddings_dataset
# with open(f'./pickleObjects/ted_codeBERT_FULL.pkl', 'wb') as f:  # open a text file
#     pickle.dump(train_embeddings_dataset, f) # serialize the list
#     f.close()

In [ ]:
train_subset_embeddings_dataset = train_dataset_subset.map(
        lambda x: {"embeddings": get_embeddings(x["whole_func_string"]).detach().cpu().numpy()[0]}
    )

train_subset_embeddings_dataset.add_faiss_index(column="embeddings")

with open(f'./pickleObjects/tsed_codeBERT_50K.pkl', 'wb') as f:  # open a text file
    pickle.dump(train_subset_embeddings_dataset, f) # serialize the list
    f.close()